In [1]:
from keras.layers import *
from keras.models import Model
from keras import regularizers
import pickle
from keras.callbacks import *
from keras.utils import to_categorical
import numpy as np
from keras import regularizers
from keras import models
import os

Using TensorFlow backend.


In [2]:
class EventPrediction(object):

    def __init__(self,window=3,filters=28,bch=25,epoch=20,train_test_split=0.8):
        self.window=window
        self.filters=filters
        self.batch=bch
        self.epc=epoch
        self.mc=ModelCheckpoint("./checkpoints"+"weights.{epoch:02d}-loss{loss:.2f}-acc{acc:.2f}-.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
        #self.es=EarlyStopping(monitor='loss', min_delta=1, patience=15, verbose=1, mode='auto')
        self.tb=TensorBoard(log_dir='./logs',  batch_size=self.batch, write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
        self.data=None
        self.split=train_test_split
        self.end=None
        self.model=None

    def load_data(self):
        choice=input("Enter number to load dataset for-\n1.large\n2.mid\n3.small")
        if(choice=='1'):
            name='large'
        elif(choice=='2'):
            name='mid'
        elif(choice=='3'):
            name='small'
        else:
            raise(Exception("Invalid input"))
        with open("./small-new-lms_vec_training.pkl","rb")as f:
            self.data=pickle.load(f)
        self.end=int(len(self.data['Y'])*self.split)
        
    def f1_loss(self, y_true, y_pred):
    
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())

        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
        return 1 - K.mean(f1)

    def load_model(self):
        longt=Input(shape=(28,),name='long')
        medt=Input(shape=(28,),name='medium')
        st=Input(shape=(None,28),name="short")
        
        hist_ind = Input(shape=(10,1),name='historical_indicators')
        
        with open("./embedding_matrix.pkl","rb")as f:
            embedding_matrix=pickle.load(f)
        
        std=Dropout(0.4)(st)
        
        lt0=Embedding(3913,100,weights=[embedding_matrix], input_length=28, trainable=False)(longt)
        
        lt1=Conv1D(self.filters, self.window, input_shape=(None,28),strides=1, padding='valid', dilation_rate=1, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(lt0)
        
        lt2=Conv1D(self.filters, 2,input_shape=(None,28), strides=1, padding='valid', dilation_rate=1, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(lt1)
        
        lt2d=Dropout(0.4)(lt2)
        
        mt0=Embedding(3913,100,weights=[embedding_matrix], input_length=28, trainable=False)(medt)
        
        mt1=Conv1D(self.filters, self.window, input_shape=(None,28),strides=1, padding='valid', dilation_rate=1, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt0)
        
        mt2=Conv1D(self.filters, 2,strides=1,input_shape=(None,28), padding='valid', dilation_rate=1, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt1)
        
        mt2d=Dropout(0.4)(mt2)
        
        ltoutput=GlobalMaxPooling1D()(lt2d)
        mtoutput=GlobalMaxPooling1D()(mt2d)
        stoutput=GlobalAveragePooling1D()(st)
        
        
        feature=concatenate([ltoutput,mtoutput,stoutput])
        h1=Dense(50,activation='relu')(feature)
        h2=Dense(10,activation='relu')(h1)
        
        hind = LSTM(units = 50, return_sequences = True, input_shape = (10, 1))(hist_ind)
        hind2d = Dropout(0.4)(hind)
        tsoutput=GlobalMaxPooling1D()(hind2d)
        
        feature2=concatenate([h2,tsoutput])
        h3=Dense(50,activation='relu')(feature2)
        h4=Dense(10,activation='relu')(h3)
        
        trend=Dense(1,activation='sigmoid',name='class')(h4)
        self.model=Model(inputs=[longt,medt,st,hist_ind],outputs=[trend])
        
        
        
        
#         trend2=Dense(1,activation='sigmoid',name='class')(h4)
#         self.model=Model(inputs=[hist_ind],outputs=[trend2])
        
        
        
        self.model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])
        self.model.summary()

    def load_saved_model(self):
        files=os.listdir()
        file_name=[i for i  in files if '.hdf5' in i]
        print("Select the saved model to use -")
        for i,j in enumerate(file_name):
            print(str(i+1)+"-"+j)
        choice=input()
        print("#"*5+"  Using saved model-"+file_name[int(choice)-1]+"  "+"#"*5)
        model=models.load_model(os.path.join(os.getcwd(),file_name[int(choice)-1]))
        print("#"*5+"  Model Loaded  "+"#"*5)
        self.model=model

    def train(self):

        self.model.fit({"long":self.data['X']['long'][:self.end],"medium":self.data['X']['medium'][:self.end],"short":self.data['X']['short'][:self.end]},
        {'class':self.data['Y'][:self.end]},epochs=self.epc,batch_size=self.batch,callbacks=None)

    def test(self,data=None):
        print("#"*5+"  Testing  "+"#"*5)
        if(data==None):
            data=self.data
        loss_metrics=self.model.evaluate({"long":data['X']['long'][self.end:],"medium":data['X']['medium'][self.end:],"short":data['X']['short'][self.end:]},
        {'class':data['Y'][self.end:]},batch_size=1)
        for i,j in enumerate(loss_metrics):
            print(self.model.metrics_names[i],j)

In [4]:
predict=EventPrediction()
predict.load_data()
predict.load_model()
# predict.load_saved_model()
predict.train()
predict.test()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
long (InputLayer)               (None, 28)           0                                            
__________________________________________________________________________________________________
medium (InputLayer)             (None, 28)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 28, 100)      391300      long[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 28, 100)      391300      medium[0][0]                     
____________________________________________________________________________________________